In [ ]:
!pip install pandas numpy scikit-learn xgboost matplotlib seaborn optuna joblib openpyxl
!pip install scikit-learn==1.0.2 # Asegura compatibilidad de IterativeImputer si es necesario

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
import os
import joblib # Para guardar y cargar el modelo

# Importaciones de Scikit-learn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.experimental import enable_iterative_imputer # Necesario para IterativeImputer
from sklearn.impute import IterativeImputer # Se mantiene por si hay otros NaNs, aunque los imputamos antes

# Importaciones de XGBoost
import xgboost as xgb

# Importaciones de Optuna
import optuna

In [3]:
# --- Configuración de rutas (AJUSTAR SEGÚN TU ENTORNO) ---
# Si estás en Google Colab y quieres usar Drive, DESCOMENTA estas líneas:
print("Montando Google Drive...")
drive.mount('/content/drive')
print("Google Drive montado.")

Montando Google Drive...
Mounted at /content/drive
Google Drive montado.


In [4]:
# --- Configuración de rutas (AJUSTAR SEGÚN TU ENTORNO) ---
# Si estás en Google Colab y quieres usar Drive, DESCOMENTA estas líneas:
print("Montando Google Drive...")
drive.mount('/content/drive')
print("Google Drive montado.")
input_file_path = '/content/drive/MyDrive/MP_20253/Datos_MediaMovil.xlsx'
#output_dir_path = '/content/drive/MyDrive/MP_20253/'

# --- 1. Cargar el DataFrame ---
print(f"Cargando datos desde: {input_file_path}")
try:
    df = pd.read_excel(input_file_path)
    print("Datos cargados exitosamente.")
except FileNotFoundError:
    print(f"Error: El archivo no se encontró en la ruta '{input_file_path}'.")
    print("Por favor, verifica la ruta y que el archivo exista. Si estás en Colab, asegúrate de montar Drive y descomentar las rutas de Drive.")
    exit()
except Exception as e:
    print(f"Error al cargar el archivo Excel: {e}")
    exit()

print("\nPrimeras 5 filas del DataFrame cargado:")
print(df.head())
print("\nInformación del DataFrame cargado:")
print(df.info())
print(f"Número total de filas en el dataset: {len(df)}")


Montando Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive montado.
Cargando datos desde: /content/drive/MyDrive/MP_20253/Datos_MediaMovil.xlsx
Datos cargados exitosamente.

Primeras 5 filas del DataFrame cargado:
   PM10_SJL_IMP  PM10_SA_IMP  PM2_5_SA_IMP  MA_PM10_SA_IMP_24h  \
0          54.6        50.89          13.8           69.423333   
1          51.5        35.14          18.3           58.648750   
2          39.3        30.89          12.9           52.925417   
3          31.3        30.14           8.5           47.282500   
4          24.9        36.51           9.9           45.407500   

   MA_PM10_SA_IMP_6h  MA_PM10_SJL_IMP_24h  MA_PM10_SJL_IMP_6h  \
0          67.200000            60.420833           49.633333   
1          59.498333            57.279167           53.150000   
2          53.008333            49.929167           52.050000   
3          47.286667 

In [5]:
# --- 2. Definir Target (y) y Características (X) para predicción 'una hora adelante' ---
# Target: PM10_SA en el siguiente periodo de tiempo (t+1)
# Se crea la columna de target desplazada una posición hacia arriba (shift(-1)).
# Esto significa que la fila actual 'i' contendrá el PM10_SA de la fila 'i+1'.
# PM10_SA(t+1) = f(Características(t))
df_original_y_values = df['PM10_SA_IMP'].copy() # Guardar los valores originales para la gráfica
df['PM10_SA_NEXT_HOUR'] = df['PM10_SA_IMP'].shift(-1)
target_column_name = 'PM10_SA_NEXT_HOUR' # Cambiado para claridad en el código

# Características (X): Todas las columnas de contaminantes (incluido el PM10_SA actual)
# Esto permite que el modelo use el PM10_SA del momento actual (t) para predecir el futuro (t+1).
features = [
    'PM10_SA_IMP', 'PM2_5_SA_IMP', 'PM10_SJL_IMP',
    'MA_PM10_SA_IMP_24h', 'MA_PM10_SA_IMP_6h',
    'MA_PM10_SJL_IMP_24h', 'MA_PM10_SJL_IMP_6h',
    'MA_PM2_5_SA_IMP_24h', 'MA_PM2_5_SA_IMP_6h'
]

# Verificar que todas las características existan
missing_features = [f for f in features if f not in df.columns]
if missing_features:
    print(f"\nError: Las siguientes características no se encontraron en el DataFrame: {missing_features}")
    print("Por favor, verifica los nombres de las columnas en tu archivo.")
    exit()

# Eliminar la última fila, ya que 'PM10_SA_NEXT_HOUR' será NaN allí
df.dropna(subset=[target_column_name], inplace=True)

X = df[features]
y = df[target_column_name]

print(f"\nVariable objetivo (y): {target_column_name}")
print(f"Número de características (X): {len(features)}")
print(f"Número total de filas después de la preparación para predicción adelante: {len(df)}")



Variable objetivo (y): PM10_SA_NEXT_HOUR
Número de características (X): 9
Número total de filas después de la preparación para predicción adelante: 82517


In [6]:
# --- 3. División de Datos (80% Train, 20% Test) - Basado en orden cronológico (ID) ---
# Se asume que el DataFrame ya está ordenado por ID, lo cual es esencial para series de tiempo.
split_idx = int(0.75 * len(df))

X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

print(f"\nDivisión de datos:")
print(f"  Tamaño del conjunto de entrenamiento: {len(X_train)} filas")
print(f"  Tamaño del conjunto de prueba: {len(X_test)} filas")


# --- 4. Escalado de Datos (MinMaxScaler) ---
print("\nEscalando datos con MinMaxScaler...")
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

# Reshape y_train y y_test para scaler_y (MinMaxScaler espera 2D array)
# .flatten() convierte de nuevo a 1D, que es lo que espera XGBoost
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1)).flatten()
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1)).flatten()

print("Escalado completado.")




División de datos:
  Tamaño del conjunto de entrenamiento: 61887 filas
  Tamaño del conjunto de prueba: 20630 filas

Escalando datos con MinMaxScaler...
Escalado completado.


In [8]:
# --- 5. Optimización de Hiperparámetros con Optuna y TimeSeriesSplit ---
print("\n--- Iniciando optimización de hiperparámetros con Optuna y TimeSeriesSplit ---")

# Optuna necesita un nombre de estudio único si se ejecuta múltiples veces
study_name = "xgboost_pm10_sa_next_hour_optimization"
# Puedes usar un RDB (SQLite) para guardar y cargar estudios, útil si se detiene el proceso
# storage = f"sqlite:///{study_name}.db" # Descomentar para usar almacenamiento

def objective(trial):
    param = {
        'objective': 'reg:squarederror', # Para problemas de regresión
        'eval_metric': 'rmse', # Métrica de evaluación para Optuna (RMSE)
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000), # Rango de árboles
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3), # Rango de tasa de aprendizaje
        'max_depth': trial.suggest_int('max_depth', 3, 10), # Profundidad máxima del árbol
        'subsample': trial.suggest_float('subsample', 0.6, 1.0), # Fracción de muestras para cada árbol
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0), # Fracción de características para cada árbol
        'gamma': trial.suggest_float('gamma', 0.0, 0.5), # Regularización mínima de pérdida requerida
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10), # Peso mínimo de la hoja
        'random_state': 42,
        'n_jobs': -1 # Usar todos los núcleos de CPU disponibles
    }

    model = xgb.XGBRegressor(**param)

    # TimeSeriesSplit para validación cruzada adecuada para series de tiempo
    tscv = TimeSeriesSplit(n_splits=5)

    rmse_scores = []
    for train_index, val_index in tscv.split(X_train_scaled):
        X_train_fold, X_val_fold = X_train_scaled[train_index], X_train_scaled[val_index]
        # CORRECCIÓN AQUÍ: y_train_scaled en lugar de y_val_fold
        y_train_fold, y_val_fold = y_train_scaled[train_index], y_train_scaled[val_index]

        model.fit(X_train_fold, y_train_fold)
        val_preds = model.predict(X_val_fold)
        rmse_scores.append(np.sqrt(mean_squared_error(y_val_fold, val_preds)))

    # Optuna busca minimizar el objetivo, por lo que devolvemos el RMSE promedio
    return np.mean(rmse_scores)


# Crear un estudio de Optuna y optimizar.
# n_trials: número de combinaciones de hiperparámetros a probar. Ajustar según el tiempo disponible.
# Puedes reducir n_trials (ej. 20-50) para pruebas más rápidas, o aumentarlo (100-200+) para una búsqueda más exhaustiva.
# Para guardar el estudio y poder reanudarlo (útil para muchos trials), descomenta las líneas 'storage'
study = optuna.create_study(direction='minimize', study_name=study_name) # , storage=storage, load_if_exists=True
study.optimize(objective, n_trials=50, show_progress_bar=True)

print("\n--- Optimización de hiperparámetros completada. ---")
print(f"Mejores hiperparámetros encontrados: {study.best_params}")
print(f"Mejor RMSE en validación cruzada: {study.best_value:.4f}")

best_hyperparams = study.best_params



[I 2025-08-01 16:34:46,511] A new study created in memory with name: xgboost_pm10_sa_next_hour_optimization



--- Iniciando optimización de hiperparámetros con Optuna y TimeSeriesSplit ---


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-08-01 16:34:50,677] Trial 0 finished with value: 0.027905351380055926 and parameters: {'n_estimators': 764, 'learning_rate': 0.058699682912631966, 'max_depth': 8, 'subsample': 0.851756124630874, 'colsample_bytree': 0.8064771252353556, 'gamma': 0.49847480614946904, 'min_child_weight': 7}. Best is trial 0 with value: 0.027905351380055926.
[I 2025-08-01 16:34:53,500] Trial 1 finished with value: 0.026995744949006438 and parameters: {'n_estimators': 736, 'learning_rate': 0.1714691190355481, 'max_depth': 10, 'subsample': 0.8623249878004555, 'colsample_bytree': 0.9598541320879266, 'gamma': 0.3290089092889492, 'min_child_weight': 1}. Best is trial 1 with value: 0.026995744949006438.
[I 2025-08-01 16:35:01,522] Trial 2 finished with value: 0.028411084436278884 and parameters: {'n_estimators': 985, 'learning_rate': 0.11706073950058757, 'max_depth': 4, 'subsample': 0.6074124187227918, 'colsample_bytree': 0.6565320645176457, 'gamma': 0.4613452182708883, 'min_child_weight': 2}. Best is tri

In [9]:
# --- 6. Entrenamiento del Modelo Final XGBoost con los Mejores Hiperparámetros ---
print("\nEntrenando el modelo final XGBoost con los mejores hiperparámetros...")
final_model = xgb.XGBRegressor(**best_hyperparams, objective='reg:squarederror', eval_metric='rmse', random_state=42, n_jobs=-1)
final_model.fit(X_train_scaled, y_train_scaled)
print("Modelo final entrenado.")


Entrenando el modelo final XGBoost con los mejores hiperparámetros...
Modelo final entrenado.


In [10]:
# --- 7. Predicción sobre todo el conjunto de datos ---
print("\n🔮 Realizando predicciones sobre todo el conjunto de datos (X)...")
X_all_scaled = scaler_X.transform(X)
y_pred_scaled = final_model.predict(X_all_scaled)

# --- 7b. Inversión del escalado ---
y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()
print("✅ Predicciones desescaladas correctamente.")


# --- 8. Agregar columna de predicciones al DataFrame original ---
df_resultado = df.copy()
df_resultado['PM10_SA_NEXT_HOUR_PRED'] = y_pred
print("📌 Columna 'PM10_SA_NEXT_HOUR_PRED' agregada al DataFrame.")




🔮 Realizando predicciones sobre todo el conjunto de datos (X)...
✅ Predicciones desescaladas correctamente.
📌 Columna 'PM10_SA_NEXT_HOUR_PRED' agregada al DataFrame.


In [13]:

# --- 9. Guardar el DataFrame con las predicciones como archivo Excel ---
output_dir_path = '/content/drive/MyDrive/MP_20253'
os.makedirs(output_dir_path, exist_ok=True)

excel_output_name = 'df_resultado_pm10_sa_MedMovil_next_hour.xlsx'
excel_output_path = os.path.join(output_dir_path, excel_output_name)

df_resultado.to_excel(excel_output_path, index=False)
print(f"📁 Archivo Excel exportado: {excel_output_path}")


# --- 10. Guardar el modelo entrenado como archivo .joblib ---
model_output_name = 'xgboost_model_pm10_sa_MedMovil_next_hour.joblib'
model_output_path = os.path.join(output_dir_path, model_output_name)

joblib.dump(final_model, model_output_path)
print(f"📦 Modelo XGBoost guardado en: {model_output_path}")


📁 Archivo Excel exportado: /content/drive/MyDrive/MP_20253/df_resultado_pm10_sa_MedMovil_next_hour.xlsx
📦 Modelo XGBoost guardado en: /content/drive/MyDrive/MP_20253/xgboost_model_pm10_sa_MedMovil_next_hour.joblib


In [12]:
# --- 7. Métricas de Evaluación ---

# Predicciones escaladas
y_train_pred_scaled = final_model.predict(X_train_scaled)
y_test_pred_scaled = final_model.predict(X_test_scaled)

# Invertir la escala de las predicciones a la escala original
y_train_pred = scaler_y.inverse_transform(y_train_pred_scaled.reshape(-1, 1)).flatten()
y_test_pred = scaler_y.inverse_transform(y_test_pred_scaled.reshape(-1, 1)).flatten()

# Asegurar que y_train y y_test también estén en el formato correcto para las métricas
y_train_actual = y_train.values
y_test_actual = y_test.values

# Función para calcular MAPE (Manejo de ceros para evitar divisiones por cero)
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    # Filtrar valores reales muy cercanos a cero para evitar inestabilidad en la división
    non_zero_mask = np.abs(y_true) > 1e-6 # Considerar 'cero' si el valor absoluto es muy pequeño
    if not np.any(non_zero_mask): # Si no hay valores no-cero válidos
        return np.nan # MAPE no es significativo

    mape = np.mean(np.abs((y_true[non_zero_mask] - y_pred[non_zero_mask]) / y_true[non_zero_mask])) * 100
    return mape

print("\n--- Métricas de Evaluación ---")

# Métricas para TRAIN
mae_train = mean_absolute_error(y_train_actual, y_train_pred)
rmse_train = np.sqrt(mean_squared_error(y_train_actual, y_train_pred))
mse_train = mean_squared_error(y_train_actual, y_train_pred)
r2_train = r2_score(y_train_actual, y_train_pred)
mape_train = mean_absolute_percentage_error(y_train_actual, y_train_pred)

print("\nConjunto de Entrenamiento:")
print(f"  MAE: {mae_train:.4f}")
print(f"  RMSE: {rmse_train:.4f}")
print(f"  MSE: {mse_train:.4f}")
print(f"  R2: {r2_train:.4f}")
print(f"  MAPE: {mape_train:.2f}%" if not np.isnan(mape_train) else "  MAPE: N/A (valores reales muy cercanos a cero)")


# Métricas para TEST
mae_test = mean_absolute_error(y_test_actual, y_test_pred)
rmse_test = np.sqrt(mean_squared_error(y_test_actual, y_test_pred))
mse_test = mean_squared_error(y_test_actual, y_test_pred)
r2_test = r2_score(y_test_actual, y_test_pred)
mape_test = mean_absolute_percentage_error(y_test_actual, y_test_pred)

print("\nConjunto de Prueba:")
print(f"  MAE: {mae_test:.4f}")
print(f"  RMSE: {rmse_test:.4f}")
print(f"  MSE: {mse_test:.4f}")
print(f"  R2: {r2_test:.4f}")
print(f"  MAPE: {mape_test:.2f}%" if not np.isnan(mape_test) else "  MAPE: N/A (valores reales muy cercanos a cero)")



--- Métricas de Evaluación ---

Conjunto de Entrenamiento:
  MAE: 10.8231
  RMSE: 16.4150
  MSE: 269.4522
  R2: 0.7972
  MAPE: 16.67%

Conjunto de Prueba:
  MAE: 7.3430
  RMSE: 12.0838
  MSE: 146.0180
  R2: 0.8092
  MAPE: 17.07%
